In [1]:
import numpy as np
import pandas as pd
import auxil_ML as aux


In [2]:
if 1:
    import importlib
    importlib.reload(aux);


In [3]:
#cat_fn = 'data/3FGL_prob_catalog_3class_newfeats.csv'
#cat_fn = '../ML_3FGL_paper_extra/data/python files/catas/3fgl_multi_cata_newfeats.csv'
cat_fn = '../SOM/3FGL_prob_catalog_3classes.csv'
cat_df3 = pd.read_csv(cat_fn, index_col=0)
cat_fn = '../SOM/3FGL_prob_catalog_2classes.csv'
cat_df2 = pd.read_csv(cat_fn, index_col=0)

#folder = '../ML_3FGL_paper/SOM'
#cat_fn = '%s/3FGL_prob_catalog.csv' % folder
cat_fn = '../SOM/3FGL_prob_catalog_2classes.csv'
cat_df_3fgl = pd.read_csv(cat_fn, index_col=0)
cat_fn = '../SOM/4FGL-DR2_prob_catalog_3classes.csv' #% folder
cat_df_4fgl = pd.read_csv(cat_fn, index_col=0)

In [22]:
nclass = 3
classes = ['AGN', 'PSR', 'OTHER'][:nclass]
algs = ['RF', 'BDT', 'NN', 'LR']

In [23]:
assoc_3FGL_in_4FGL_mask = [(type(name) is str) for name in list(cat_df_4fgl['ASSOC_FGL'])]
index0 = [name for name in cat_df_4fgl.index[assoc_3FGL_in_4FGL_mask] 
         if cat_df_4fgl['ASSOC_FGL'].loc[name].startswith('3FGL') 
         and cat_df_4fgl['ASSOC_FGL'].loc[name] in cat_df_3fgl.index
         and cat_df_3fgl['Category_3FGL'].loc[cat_df_4fgl['ASSOC_FGL'].loc[name]] == 'UNAS']

index = [name for name in index0 if cat_df_4fgl['Category_4FGL'].loc[name] != 'UNAS']
index_2class = [name for name in index if cat_df_4fgl['Category_4FGL'].loc[name] 
                in ['PSR', 'AGN']]
index_3class = [name for name in index if cat_df_4fgl['Category_4FGL'].loc[name] 
                in ['PSR', 'AGN', 'OTHER']]
if nclass == 2:
    index = index_2class
elif nclass == 3:
    index = index_3class
index_unas = [name for name in index0 if cat_df_4fgl['Category_4FGL'].loc[name] == 'UNAS']
index_other = [name for name in index if cat_df_4fgl['Category_4FGL'].loc[name] == 'OTHER']

assoc_3FGL_in_4FGL_df = pd.DataFrame(index=cat_df_4fgl['ASSOC_FGL'][index])
assoc_3FGL_in_4FGL_df['Category_4FGL'] = np.array(cat_df_4fgl['Category_4FGL'].loc[index])
psr_mask_4FGL = (assoc_3FGL_in_4FGL_df['Category_4FGL'] == 'PSR')
agn_mask_4FGL = (assoc_3FGL_in_4FGL_df['Category_4FGL'] == 'AGN')
other_mask_4FGL = (assoc_3FGL_in_4FGL_df['Category_4FGL'] == 'OTHER')


In [24]:
# missing values
index_miss = [name for name in index if 
              cat_df_3fgl['Missing_Values_Flag'].loc[cat_df_4fgl['ASSOC_FGL'].loc[name]] == 1]
print('Sources with missing values in 3FGL, which have associations in 4FGL-DR2:')
print(cat_df_4fgl[['Category_4FGL', 'Category_Prob']].loc[index_miss])
#print(cat_df_4fgl['ASSOC_FGL'].loc[index_miss])
print(cat_df_3fgl['Category_Prob'].loc[cat_df_4fgl['ASSOC_FGL'].loc[index_miss]])

Sources with missing values in 3FGL, which have associations in 4FGL-DR2:
                  Category_4FGL Category_Prob
Source_Name_4FGL                             
4FGL J0121.8-3916           AGN           AGN
Source_Name_3FGL
3FGL J0121.8-3917    AGN
Name: Category_Prob, dtype: object


In [25]:
print('Unas in 3FGL, assoc in 4FGL-DR2:', len(index))
print('Unas in 3FGL, unas in 4FGL-DR2:', len(index_unas))
print('Unas in 3FGL, AGN or PSR in 4FGL-DR2:', len(index_2class))
print('Unas in 3FGL, OTHER in 4FGL-DR2:', len(index_other))
print('Ratio of OTHER to assoc in 4FGL-DR2:', len(index_other)/len(index))

Unas in 3FGL, assoc in 4FGL-DR2: 340
Unas in 3FGL, unas in 4FGL-DR2: 303
Unas in 3FGL, AGN or PSR in 4FGL-DR2: 303
Unas in 3FGL, OTHER in 4FGL-DR2: 37
Ratio of OTHER to assoc in 4FGL-DR2: 0.10882352941176471


In [26]:
other_4fgl = sum(cat_df_4fgl['Category_4FGL'] == 'OTHER')
assoc_4fgl = sum(cat_df_4fgl['Category_4FGL'] != 'UNAS')
print('OTHER in 4FGL-DR2: %i' % other_4fgl)
print('Assoc in 4FGL-DR2: %i' % assoc_4fgl)
print('Fraction: %.4f' % (other_4fgl/assoc_4fgl))

OTHER in 4FGL-DR2: 347
Assoc in 4FGL-DR2: 4121
Fraction: 0.0842


In [27]:
cat_df_3fgl_i = cat_df_3fgl.loc[assoc_3FGL_in_4FGL_df.index]
cat_df3_i = cat_df3.loc[assoc_3FGL_in_4FGL_df.index]
cat_df2_i = cat_df2.loc[assoc_3FGL_in_4FGL_df.index]


In [28]:
psr_mask_prob = (cat_df_3fgl_i['Category_Prob'] == 'PSR')
agn_mask_prob = (cat_df_3fgl_i['Category_Prob'] == 'AGN')
mix_mask_prob = (cat_df_3fgl_i['Category_Prob'] == 'MIXED')

psr_mask_prob2 = (cat_df2_i['Category_Prob'] == 'PSR')
agn_mask_prob2 = (cat_df2_i['Category_Prob'] == 'AGN')
mix_mask_prob2 = (cat_df2_i['Category_Prob'] == 'MIXED')

psr_mask_prob3 = (cat_df3_i['Category_Prob'] == 'PSR')
agn_mask_prob3 = (cat_df3_i['Category_Prob'] == 'AGN')
other_mask_prob3 = (cat_df3_i['Category_Prob'] == 'OTHER')
mix_mask_prob3 = (cat_df3_i['Category_Prob'] == 'MIXED')

In [29]:
print('2-class classification in paper')
print('# PSR classified as PSR', np.sum(psr_mask_4FGL & psr_mask_prob))
print('# PSR classified as AGN', np.sum(psr_mask_4FGL & agn_mask_prob))
print('# PSR classified as mixed', np.sum(psr_mask_4FGL & mix_mask_prob))
print('# AGN classified as AGN', np.sum(agn_mask_4FGL & agn_mask_prob))
print('# AGN classified as PSR', np.sum(agn_mask_4FGL & psr_mask_prob))
print('# AGN classified as mixed', np.sum(agn_mask_4FGL & mix_mask_prob))
print('# OTHER classified as AGN', np.sum(other_mask_4FGL & agn_mask_prob))
print('# OTHER classified as PSR', np.sum(other_mask_4FGL & psr_mask_prob))
print('# OTHER classified as mixed', np.sum(other_mask_4FGL & mix_mask_prob))

2-class classification in paper
# PSR classified as PSR 23
# PSR classified as AGN 10
# PSR classified as mixed 12
# AGN classified as AGN 223
# AGN classified as PSR 5
# AGN classified as mixed 30
# OTHER classified as AGN 8
# OTHER classified as PSR 6
# OTHER classified as mixed 23


In [42]:
# calculations for Table 4 source numbers and precision / recall 3FGL vs 4FGL comparison
print('2-class classification')
print('# AGN classified as AGN', np.sum(agn_mask_4FGL & agn_mask_prob2 ))
print('# AGN classified as PSR', np.sum(agn_mask_4FGL & psr_mask_prob2 ))
print('# AGN classified as mixed', np.sum(agn_mask_4FGL & mix_mask_prob2 ))
print('AGN precision: %.2f' % (np.sum(agn_mask_4FGL & agn_mask_prob2)/np.sum(agn_mask_prob2)))
print('AGN recall: %.2f' % (np.sum(agn_mask_4FGL & agn_mask_prob2)/np.sum(agn_mask_4FGL)))
print('# PSR classified as AGN', np.sum(psr_mask_4FGL & agn_mask_prob2 ))
print('# PSR classified as PSR', np.sum(psr_mask_4FGL & psr_mask_prob2 ))
print('# PSR classified as mixed', np.sum(psr_mask_4FGL & mix_mask_prob2 ))
print('PSR precision: %.2f' % (np.sum(psr_mask_4FGL & psr_mask_prob2)/np.sum(psr_mask_prob2)))
print('PSR recall: %.2f' % (np.sum(psr_mask_4FGL & psr_mask_prob2)/np.sum(psr_mask_4FGL)))

print('# OTHER classified as AGN', np.sum(other_mask_4FGL & agn_mask_prob2))
print('# OTHER classified as PSR', np.sum(other_mask_4FGL & psr_mask_prob2))
print('# OTHER classified as mixed', np.sum(other_mask_4FGL & mix_mask_prob2))

2-class classification
# AGN classified as AGN 223
# AGN classified as PSR 5
# AGN classified as mixed 30
AGN precision: 0.93
AGN recall: 0.86
# PSR classified as AGN 10
# PSR classified as PSR 23
# PSR classified as mixed 12
PSR precision: 0.68
PSR recall: 0.51
# OTHER classified as AGN 8
# OTHER classified as PSR 6
# OTHER classified as mixed 23


In [44]:
# calculations for Table 10 source numbers and precision / recall 3FGL vs 4FGL comparison
print('3-class classification')
print('# AGN classified as AGN', np.sum(agn_mask_4FGL & agn_mask_prob3 ))
print('# AGN classified as PSR', np.sum(agn_mask_4FGL & psr_mask_prob3 ))
print('# AGN classified as OTHER', np.sum(agn_mask_4FGL & other_mask_prob3 ))
print('# AGN classified as mixed', np.sum(agn_mask_4FGL & mix_mask_prob3 ))
print('AGN precision: %.2f' % (np.sum(agn_mask_4FGL & agn_mask_prob3)/np.sum(agn_mask_prob3)))
print('AGN recall: %.2f' % (np.sum(agn_mask_4FGL & agn_mask_prob3)/np.sum(agn_mask_4FGL)))
print('# PSR classified as AGN', np.sum(psr_mask_4FGL & agn_mask_prob3 ))
print('# PSR classified as PSR', np.sum(psr_mask_4FGL & psr_mask_prob3 ))
print('# PSR classified as OTHER', np.sum(psr_mask_4FGL & other_mask_prob3 ))
print('# PSR classified as mixed', np.sum(psr_mask_4FGL & mix_mask_prob3 ))
print('PSR precision: %.2f' % (np.sum(psr_mask_4FGL & psr_mask_prob3)/np.sum(psr_mask_prob3)))
print('PSR recall: %.2f' % (np.sum(psr_mask_4FGL & psr_mask_prob3)/np.sum(psr_mask_4FGL)))
print('# OTHER classified as AGN', np.sum(other_mask_4FGL & agn_mask_prob3))
print('# OTHER classified as PSR', np.sum(other_mask_4FGL & psr_mask_prob3))
print('# OTHER classified as OTHER', np.sum(other_mask_4FGL & other_mask_prob3))
print('# OTHER classified as mixed', np.sum(other_mask_4FGL & mix_mask_prob3))
print('OTHER precision: %.2f' % (np.sum(other_mask_4FGL & other_mask_prob3)/np.sum(other_mask_prob3)))
print('OTHER recall: %.2f' % (np.sum(other_mask_4FGL & other_mask_prob3)/np.sum(other_mask_4FGL)))


3-class classification
# AGN classified as AGN 238
# AGN classified as PSR 2
# AGN classified as OTHER 1
# AGN classified as mixed 17
AGN precision: 0.93
AGN recall: 0.92
# PSR classified as AGN 12
# PSR classified as PSR 17
# PSR classified as OTHER 0
# PSR classified as mixed 16
PSR precision: 0.71
PSR recall: 0.38
# OTHER classified as AGN 6
# OTHER classified as PSR 5
# OTHER classified as OTHER 8
# OTHER classified as mixed 18
OTHER precision: 0.89
OTHER recall: 0.22


In [41]:
# calculations for Tables 2 and 9, comparison with 4FGL columns
true_labels = assoc_3FGL_in_4FGL_df['Category_4FGL']
classif_dict = {'paper': cat_df_3fgl_i, '2 class': cat_df2_i, '3 class': cat_df3_i}
for cname, cdf in classif_dict.items():
    print(cname)
    if cname == 'paper':
        algs_loc = algs + [alg + '_O' for alg in algs]
    else:
        algs_loc = algs + [alg + '_O' for alg in algs]
    for alg in algs_loc:
        prob_labels = aux.get_prob_class(cdf, [alg], classes)['Category_Prob']
        acc_all = 100 * aux.accuracy(true_labels, prob_labels)
        #print(sum(prob_labels == 'OTHER'), sum(true_labels == 'OTHER'))
        print('%s: %i / %i = %.2f percent' % \
              (alg, sum(prob_labels == true_labels), len(true_labels), acc_all))
        #print(sum((prob_labels == 'OTHER') & (prob_labels == true_labels)))
        #print('%s, %.2f percent' % (alg, acc_all))

paper
RF: 276 / 340 = 81.18 percent
BDT: 274 / 340 = 80.59 percent
NN: 273 / 340 = 80.29 percent
LR: 274 / 340 = 80.59 percent
RF_O: 271 / 340 = 79.71 percent
BDT_O: 278 / 340 = 81.76 percent
NN_O: 264 / 340 = 77.65 percent
LR_O: 258 / 340 = 75.88 percent
2 class
RF: 276 / 340 = 81.18 percent
BDT: 274 / 340 = 80.59 percent
NN: 273 / 340 = 80.29 percent
LR: 274 / 340 = 80.59 percent
RF_O: 271 / 340 = 79.71 percent
BDT_O: 278 / 340 = 81.76 percent
NN_O: 264 / 340 = 77.65 percent
LR_O: 258 / 340 = 75.88 percent
3 class
RF: 289 / 340 = 85.00 percent
BDT: 283 / 340 = 83.24 percent
NN: 284 / 340 = 83.53 percent
LR: 283 / 340 = 83.24 percent
RF_O: 289 / 340 = 85.00 percent
BDT_O: 290 / 340 = 85.29 percent
NN_O: 278 / 340 = 81.76 percent
LR_O: 283 / 340 = 83.24 percent
